In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [29]:
class MF(nn.Module):
    def __init__(self, num_users, num_items, emb_size=100):
        super(MF, self).__init__()
        self.user_emb = nn.Embedding(num_users, emb_size)
        self.item_emb = nn.Embedding(num_items, emb_size)
        # initializing our matrices with a positive number generally will yield better results
        self.user_emb.weight.data.uniform_(0, 0.5)
        self.item_emb.weight.data.uniform_(0, 0.5)
    def forward(self, u, v):
        u = self.user_emb(u)
        v = self.item_emb(v)
        return (u*v).sum(1)  # taking the dot product

In [31]:
df = pd.read_csv('./ml-latest-small/ratings.csv', sep=',', header=None)

arr = df.to_numpy()
val = arr[:, 0] # first column
new_a = np.delete(val, 0).tolist()
num_users = len(set(new_a))
# print(num_users) # 610

val2 = arr[:, 1] # second column
movies = np.delete(val2, 0).tolist()
num_items = len(set(movies))

# print(num_items) #9724


model = MF(num_users, num_items, emb_size=100)

In [17]:
train_df, valid_df = train_test_split(df, test_size=0.25)
# resetting indices to avoid indexing errors
train_df = train_df.reset_index(drop=True)
test_df = valid_df.reset_index(drop=True)

In [18]:
def train_epocs(model, epochs=10, lr=0.01, wd=0.0):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
    model.train()
    for i in range(epochs):
        usernames = torch.LongTensor(train_df.userId.values)
        movie_titles = torch.LongTensor(train_df.movieId.values)
        ratings = torch.FloatTensor(train_df.rating.values)
        y_hat = model(usernames, movie_titles)
        loss = F.mse_loss(y_hat, ratings)
        optimizer.zero_grad()  # reset gradient
        loss.backward()
        optimizer.step()
        print(loss.item())
    test(model)

In [19]:
def test(model):
    model.eval()
    usernames = torch.LongTensor(train_df.userId.values)
    movie_titles = torch.LongTensor(train_df.movieId.values)
    ratings = torch.FloatTensor(train_df.rating.values)
    y_hat = model(usernames, movie_titles)
    loss = F.mse_loss(y_hat, ratings)
    print("test loss %.3f " % loss.item())

In [22]:
arr = df.to_numpy()
val = arr[:, 1] # second column
new_a = np.delete(val, 0)
numbers = [ int(x) for x in new_a ]

user = torch.tensor([10])
games = torch.tensor(numbers)

predictions = model(user, games).tolist()
print(predictions)

IndexError: index out of range in self